In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)

import numpy as np

import geopandas as gpd
import fiona

import datetime

In [2]:
sheets_dict = pd.read_excel('ui_claims_county_industry.xlsx', sheet_name=None,skiprows=2,skipfooter=3)

df = pd.DataFrame()
for name, sheet in sheets_dict.items():
    sheet['County'] = name + 'County'
    sheet = sheet.rename(columns=lambda x: x.split('\n')[-1])
    df = df.append(sheet)

df.reset_index(inplace=True, drop=True)

In [3]:
del df['Unnamed: 2']
del df['Unnamed: 4']

In [4]:
df = df.melt(id_vars=["County", "NAICS", "INDUSTRY NAME"], 
             var_name="Date", 
             value_name="Claims")

df['Claims'] = df['Claims'].replace(',', '', regex=True)

df['Claims'] = df['Claims'].astype(int)

In [5]:
warns = pd.read_excel('warnLogs_200720.xlsx')
warns['County'] = warns['County'] + ' County'
warns['County'] = warns['County'].str.strip()

In [6]:
codes = pd.read_excel('../../data/geographies/regional-commissions.xlsx', usecols='A,B,C')
codes['County'] = codes['County'] + ' County'
codes['County'] = codes['County'].astype(str)

msa = pd.read_excel('../../data/geographies/cbsa.xlsx', sheet_name='cbsa', usecols='A,D,H,I')
msa = msa.loc[msa['State'] == 'Georgia']

In [7]:
df = pd.merge(df,codes,how='left',on='County')


In [8]:
df = pd.merge(df,msa,how='left',on='County')
warns = pd.merge(warns,codes,how='left',on='County')
warns = pd.merge(warns,msa,how='left',on='County')

In [9]:
today = np.datetime64('today')
lastYear = today - 365

warnsCurr = warns[(warns['Date'] > '2020-03-01') & (warns['Date'] <= today)]
warnsCurr['Period'] = 'CurrentWarns'
warnsLast = warns[(warns['Date'] > '2019-03-01') & (warns['Date'] <= lastYear)]
warnsLast['Period'] = 'Last Period'
warnPeriods = warnsCurr.append([warnsLast])

warnPeriods = warnPeriods[['ID','Company name','Est. Impact','Date','Period']]

warns = pd.merge(warns,warnPeriods,how='left',on=['ID','Company name','Est. Impact','Date'])

warns['Period'].fillna('NA',inplace=True)

/Users/mmainza/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/mmainza/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [10]:
warns['State'] = 'Georgia'

In [11]:
countyWarns = warns.groupby(['County','Period']).size().reset_index()
countyWarns = countyWarns.pivot_table(0, ['County'], 'Period').reset_index(drop=False)
countyWarns.rename(columns={'County':'Area'}, inplace=True)
msaWarns = warns.groupby(['CBSA Title','Period']).size().reset_index()
msaWarns = msaWarns.pivot_table(0, ['CBSA Title'], 'Period').reset_index(drop=False)
msaWarns.rename(columns={'CBSA Title':'Area'}, inplace=True)
rcWarns = warns.groupby(['Regional Commission','Period']).size().reset_index()
rcWarns = rcWarns.pivot_table(0, ['Regional Commission'], 'Period').reset_index(drop=False)
rcWarns.rename(columns={'Regional Commission':'Area'}, inplace=True)
stateWarns = warns.groupby(['State','Period']).size().reset_index()
stateWarns = stateWarns.pivot_table(0, ['State'], 'Period').reset_index(drop=False)
stateWarns.rename(columns={'State':'Area'}, inplace=True)
del warns['State']

areaWarns = stateWarns.append([rcWarns,msaWarns,countyWarns])
areaWarns.reset_index(drop=True, inplace=True)

areaWarns.fillna(0, inplace=True)

In [12]:
areaWarns = areaWarns[['Area','CurrentWarns']]
areaWarns.fillna(0, inplace=True)

In [13]:
labor = pd.read_excel('county_laborForce.xlsx')
labor.dropna(subset=['title'], inplace=True)
labor = labor.loc[labor['Month'] == 2.0]
labor['Month'] = labor['Month'] + 1
labor['Date'] = pd.to_datetime(labor.Month.astype(int).astype(str) + '-7' + '-2020', format = '%m-%d-%Y')
labor['Date'] = labor['Date'].dt.strftime('%m/%d/%Y')
labor['title'] = labor['title'].replace(', GA', '', regex=True)

In [14]:
febLabor = labor[['title','Date','emp','unemp','laborforce']]

In [15]:
febLabor.rename(columns={'title'      : 'County',
                           'emp'        : 'FebEmployed',
                           'unemp'      : 'FebUnemployed',
                           'laborforce' : 'FebLaborForce'}, inplace=True)
febLabor.reset_index(drop=True, inplace=True)

/Users/mmainza/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [16]:
# create separate frame of total claims by county, not industry
countyClaims = df.groupby(['area_code','County','Date']).agg({'Claims':'sum'}).reset_index()

In [17]:
countyClaims.drop_duplicates(inplace=True)

In [18]:
countyClaims.head()

,area_code,County,Date,Claims
0,13001,Appling County,03/07/2020,5
1,13001,Appling County,03/14/2020,6
2,13001,Appling County,03/21/2020,5
3,13001,Appling County,03/28/2020,143
4,13001,Appling County,04/04/2020,322


In [19]:
febLabor.head()

,County,Date,FebEmployed,FebUnemployed,FebLaborForce
0,Appling County,03/07/2020,9534.0,363.0,9897.0
1,Atkinson County,03/07/2020,4994.0,154.0,5148.0
2,Bacon County,03/07/2020,4867.0,190.0,5057.0
3,Baker County,03/07/2020,1172.0,57.0,1229.0
4,Baldwin County,03/07/2020,17013.0,797.0,17810.0


In [20]:
countyClaims = pd.merge(countyClaims,febLabor,how='left',on=['County','Date'])

countyClaims.FebEmployed.fillna(method='ffill', inplace=True)
countyClaims.FebUnemployed.fillna(method='ffill', inplace=True)
countyClaims.FebLaborForce.fillna(method='ffill', inplace=True)

countyClaims['CumClaims'] = countyClaims.groupby(['area_code','County'])['Claims'].apply(lambda x: x.cumsum())

In [21]:
countyClaims['PctFebLabor'] = ( countyClaims.CumClaims / countyClaims.FebLaborForce ) * 100
countyClaims.PctFebLabor = countyClaims.PctFebLabor.round(1)

In [22]:
# do this for the whole state
febLaborState = febLabor.groupby('Date') \
                            .agg({'FebEmployed':'sum','FebUnemployed' : 'sum','FebLaborForce' : 'sum'}) \
                            .reset_index()

stateClaims = countyClaims[['County','Date','Claims']]

stateClaims = stateClaims.groupby('Date').agg({'Claims':'sum'}).reset_index()

stateClaims = pd.merge(stateClaims, febLaborState, how='left', on='Date')

stateClaims.FebEmployed.fillna(method='ffill', inplace=True)
stateClaims.FebUnemployed.fillna(method='ffill', inplace=True)
stateClaims.FebLaborForce.fillna(method='ffill', inplace=True)

In [23]:
stateClaims['State'] = 'Georgia'

In [24]:
stateClaims['CumClaims'] = stateClaims.groupby(['State'])['Claims'].apply(lambda x: x.cumsum())

In [25]:
stateClaims['PctFebLabor'] = ( stateClaims.CumClaims / stateClaims.FebLaborForce ) * 100
stateClaims.PctFebLabor = stateClaims.PctFebLabor.round(1)

In [26]:
stateClaims = stateClaims[['State','Date','Claims','FebEmployed','FebUnemployed','FebLaborForce',
                           'CumClaims','PctFebLabor']]

In [27]:
# now do the same with msa claims

In [28]:
febLaborMsa = pd.merge(febLabor, msa, how='left', on='County')

In [29]:
febLaborMsa.dropna(subset=['CBSA Code'], inplace=True)

In [30]:
febLaborMsa = febLaborMsa.groupby(['CBSA Title','Date']) \
                             .agg({'FebEmployed':'sum','FebUnemployed' : 'sum','FebLaborForce' : 'sum'}) \
                             .reset_index()

In [31]:
msaClaims = countyClaims[['County','Date','Claims']]
msaClaims = pd.merge(msaClaims, msa, how='left', on='County')

msaClaims.dropna(subset=['CBSA Code'], inplace=True)

msaClaims = msaClaims.groupby(['CBSA Title','Date']) \
                             .agg({'Claims':'sum'}) \
                             .reset_index()

msaClaims = pd.merge(msaClaims,febLaborMsa,how='left',on=['CBSA Title','Date'])


msaClaims.FebEmployed.fillna(method='ffill', inplace=True)
msaClaims.FebUnemployed.fillna(method='ffill', inplace=True)
msaClaims.FebLaborForce.fillna(method='ffill', inplace=True)

msaClaims['CumClaims'] = msaClaims.groupby(['CBSA Title'])['Claims'].apply(lambda x: x.cumsum())

msaClaims['PctFebLabor'] = ( msaClaims.CumClaims / msaClaims.FebLaborForce ) * 100
msaClaims.PctFebLabor = msaClaims.PctFebLabor.round(1)

In [32]:
# now repeat with regional commissions

In [33]:
febLaborRc = pd.merge(febLabor, codes, how='left', on='County')

In [34]:
febLaborRc = febLaborRc.groupby(['Regional Commission','Date']) \
                             .agg({'FebEmployed':'sum','FebUnemployed' : 'sum','FebLaborForce' : 'sum'}) \
                             .reset_index()

In [35]:
rcClaims = countyClaims[['County','Date','Claims']]
rcClaims = pd.merge(rcClaims, codes, how='left', on='County')

rcClaims = rcClaims.groupby(['Regional Commission','Date']) \
                             .agg({'Claims':'sum'}) \
                             .reset_index()

rcClaims = pd.merge(rcClaims,febLaborRc,how='left',on=['Regional Commission','Date'])


rcClaims.FebEmployed.fillna(method='ffill', inplace=True)
rcClaims.FebUnemployed.fillna(method='ffill', inplace=True)
rcClaims.FebLaborForce.fillna(method='ffill', inplace=True)

rcClaims['CumClaims'] = rcClaims.groupby(['Regional Commission'])['Claims'].apply(lambda x: x.cumsum())

rcClaims['PctFebLabor'] = ( rcClaims.CumClaims / rcClaims.FebLaborForce ) * 100
rcClaims.PctFebLabor = rcClaims.PctFebLabor.round(1)

In [36]:
del countyClaims['area_code']

msaClaims.rename(columns={'CBSA Title' : 'Area'}, inplace=True)
rcClaims.rename(columns={'Regional Commission' : 'Area'}, inplace=True)
countyClaims.rename(columns={'County' : 'Area'}, inplace=True)
stateClaims.rename(columns={'State' : 'Area'}, inplace=True)

In [37]:
# stack on top of each other and write out as a single data frame for line and bar charts

In [38]:
data = stateClaims.append([countyClaims,rcClaims,msaClaims])

In [39]:
data.reset_index(drop=True, inplace=True)

In [40]:
data = data[['Area','Date','Claims','FebLaborForce','CumClaims','PctFebLabor']]

In [41]:
dataCum = data.copy()

In [42]:
countyClaims=countyClaims.groupby('Area').apply(lambda x: x.reset_index(drop=True)).drop('Area',axis=1).reset_index()
dateMax = countyClaims.level_1.max()
countyClaims = countyClaims.loc[countyClaims['level_1'] == dateMax]
countyClaims.reset_index(drop=True, inplace=True)
countyShape = gpd.read_file("spatial/ga-counties.geojson")
countyClaims = pd.merge(countyClaims, codes, how='left', left_on='Area', right_on='County')
countyClaims.area_code = countyClaims.area_code.astype(str)

In [43]:
countyClaims = pd.merge(countyClaims,areaWarns,how='left',on='Area')

In [44]:
countyShape = pd.merge(countyShape, countyClaims, how='left', left_on='GEOID', right_on='area_code')
countyShapes = countyShape[['area_code','County','CumClaims','FebLaborForce','PctFebLabor','CurrentWarns','geometry']]

In [45]:
countyShapes = gpd.GeoDataFrame(countyShapes, geometry='geometry')
countyPoints = countyShapes.copy()
countyPoints['geometry'] = countyPoints['geometry'].centroid

In [46]:
# now reshape the industry claims data to rollup by county and industry

df['State'] = 'Georgia'

df.rename(columns={'INDUSTRY NAME':'Industry'}, inplace=True)

industryCountyClaims = df.groupby(['County','Industry']) \
                             .agg({'Claims':'sum'}) \
                             .reset_index() \
                             .rename(columns={'County':'Area'})

industryCountyClaims.sort_values(['Area','Claims'], ascending=[True,False], inplace=True)

industryRcClaims = df.groupby(['Regional Commission','Industry']) \
                             .agg({'Claims':'sum'}) \
                             .reset_index() \
                             .rename(columns={'Regional Commission':'Area'})

industryRcClaims.sort_values(['Area','Claims'], ascending=[True,False], inplace=True)

industryMsaClaims = df.groupby(['CBSA Title','Industry']) \
                             .agg({'Claims':'sum'}) \
                             .reset_index() \
                             .rename(columns={'CBSA Title':'Area'})

industryMsaClaims.sort_values(['Area','Claims'], ascending=[True,False], inplace=True)


industryStateClaims = df.groupby(['State','Industry']) \
                             .agg({'Claims':'sum'}) \
                             .reset_index() \
                             .rename(columns={'State':'Area'})

industryStateClaims.sort_values(['Area','Claims'], ascending=[True,False], inplace=True)

del df['State']

In [47]:
# append these together and reindex them
industryClaims = industryStateClaims.append([industryCountyClaims,industryRcClaims,industryMsaClaims])

industryClaims.reset_index(drop=True, inplace=True)

industryClaims['Ind'] = np.where(industryClaims.Industry.str.contains("Accommodation"), "Food Services",
                        np.where(industryClaims.Industry.str.contains("Waste"), "Support, Waste Management",
                        np.where(industryClaims.Industry.str.contains("Agriculture"), "Agriculture",
                        np.where(industryClaims.Industry.str.contains("Entertainment"), "Entertainment",
                        np.where(industryClaims.Industry.str.contains("Construction"), "Construction",
                        np.where(industryClaims.Industry.str.contains("Educational"), "Education",
                        np.where(industryClaims.Industry.str.contains("Finance"), "Finance",
                        np.where(industryClaims.Industry.str.contains("Health"), "Health Care",
                        np.where(industryClaims.Industry.str.contains("Information"), "Information",
                        np.where(industryClaims.Industry.str.contains("Management"), "Enterprise Management",
                        np.where(industryClaims.Industry.str.contains("Manufacturing"), "Manufacturing",
                        np.where(industryClaims.Industry.str.contains("Mining"), "Oil & Gas",
                        np.where(industryClaims.Industry.str.contains("Other"), "Other",
                        np.where(industryClaims.Industry.str.contains("Technical"), "Technical Services",
                        np.where(industryClaims.Industry.str.contains("Public"), "Public Admin",
                        np.where(industryClaims.Industry.str.contains("Real Estate"), "Real Estate",
                        np.where(industryClaims.Industry.str.contains("Retail Trade"), "Retail",
                        np.where(industryClaims.Industry.str.contains("Transportation"), "Transportation",
                        np.where(industryClaims.Industry.str.contains("Unclassified"), "Unclassified",
                        np.where(industryClaims.Industry.str.contains("Utilities"), "Utilities",
                        np.where(industryClaims.Industry.str.contains("Wholesale"), "Wholesale Trade","ts")))))))))))))))))))))

In [48]:
industryClaims = industryClaims[['Area','Industry','Ind','Claims']]

In [49]:
# now get the KPIs you need in a json file
dataK = data.copy()
dataK=dataK.groupby('Area').apply(lambda x: x.reset_index(drop=True)).drop('Area',axis=1).reset_index()
dateMax = dataK.level_1.max()
dataK = dataK.loc[dataK['level_1'] == dateMax]

In [50]:
dataK.sort_values('Claims', ascending=False, inplace=True)

In [51]:
dataK.head()

,Area,level_1,Date,Claims,FebLaborForce,CumClaims,PctFebLabor
1078,Georgia,12,05/30/2020,147237,5187308.0,2279729,43.9
90,"Atlanta-Sandy Springs-Alpharetta, GA",12,05/30/2020,92112,3139975.0,1442230,45.9
12,ARC,12,05/30/2020,75053,2474606.0,1170926,47.3
1052,Fulton County,12,05/30/2020,18521,565766.0,305813,54.1
1182,Gwinnett County,12,05/30/2020,13650,497011.0,221431,44.6


In [52]:
dataK = pd.merge(dataK,areaWarns,how='left',on='Area')

In [53]:
dataK = dataK[['Area','FebLaborForce','CumClaims','PctFebLabor','CurrentWarns']]

In [54]:
dataK.set_index('Area', inplace=True)

In [55]:
dataK['CurrentWarns'].fillna(0, inplace=True)
countyShapes['CurrentWarns'].fillna(0, inplace=True)
countyPoints['CurrentWarns'].fillna(0, inplace=True)

In [56]:
dataK.FebLaborForce = dataK.FebLaborForce.astype(int)
dataK.CurrentWarns = dataK.CurrentWarns.astype(int)
dataK.CumClaims = dataK.CumClaims.astype(int)

In [57]:
def human_format(num):
    num = float('{:.3g}'.format(num))
    magnitude = 0
    while abs(num) >= 1000:
        magnitude += 1
        num /= 1000.0
    return '{}{}'.format('{:f}'.format(num).rstrip('0').rstrip('.'), ['', 'K', 'M', 'B', 'T'][magnitude])

dataK['CumClaims'] = dataK['CumClaims'].apply(lambda x: human_format(x))
dataK['CurrentWarns'] = dataK['CurrentWarns'].apply(lambda x: human_format(x))
dataK['FebLaborForce'] = dataK['FebLaborForce'].apply(lambda x: human_format(x))

In [58]:
dataK.head()

,FebLaborForce,CumClaims,PctFebLabor,CurrentWarns
Area,,,,
Georgia,5.19M,2.28M,43.9,363
"Atlanta-Sandy Springs-Alpharetta, GA",3.14M,1.44M,45.9,215
ARC,2.47M,1.17M,47.3,188
Fulton County,566K,306K,54.1,97
Gwinnett County,497K,221K,44.6,23


In [59]:
# convert kpi to strings with correct format for display purposes
dataK['PctFebLabor'] = dataK['PctFebLabor'].map('{:,.1f}'.format) + '%'

In [60]:
# now process the data file from which d3 will generate charts to only include necessary columns, etc.
data["date"] = pd.to_datetime(data["Date"]).dt.strftime('%b %d')
data = data[['Area','Date','date','Claims']]

In [61]:
industryClaims['Ind'] = np.where(industryClaims.Industry.str.contains("Accommodation"), "Food Services",
                        np.where(industryClaims.Industry.str.contains("Waste"), "Waste Management",
                        np.where(industryClaims.Industry.str.contains("Agriculture"), "Agriculture",
                        np.where(industryClaims.Industry.str.contains("Entertainment"), "Entertainment",
                        np.where(industryClaims.Industry.str.contains("Construction"), "Construction",
                        np.where(industryClaims.Industry.str.contains("Educational"), "Education",
                        np.where(industryClaims.Industry.str.contains("Finance"), "Finance",
                        np.where(industryClaims.Industry.str.contains("Health"), "Health Care",
                        np.where(industryClaims.Industry.str.contains("Information"), "Information",
                        np.where(industryClaims.Industry.str.contains("Management"), "Enterprise Management",
                        np.where(industryClaims.Industry.str.contains("Manufacturing"), "Manufacturing",
                        np.where(industryClaims.Industry.str.contains("Mining"), "Oil & Gas",
                        np.where(industryClaims.Industry.str.contains("Other"), "Other",
                        np.where(industryClaims.Industry.str.contains("Technical"), "Technical Services",
                        np.where(industryClaims.Industry.str.contains("Public"), "Public Admin",
                        np.where(industryClaims.Industry.str.contains("Real Estate"), "Real Estate",
                        np.where(industryClaims.Industry.str.contains("Retail Trade"), "Retail",
                        np.where(industryClaims.Industry.str.contains("Transportation"), "Transportation",
                        np.where(industryClaims.Industry.str.contains("Unclassified"), "Unclassified",
                        np.where(industryClaims.Industry.str.contains("Utilities"), "Utilities",
                        np.where(industryClaims.Industry.str.contains("Wholesale"), "Wholesale Trade","ts")))))))))))))))))))))

In [62]:
appWarns = warns[warns['Date'] >= '2020-01-01']

In [63]:
appWarns = appWarns[['Date','Company name','City','ZIP','County','Est. Impact','Regional Commission','CBSA Title']]
appWarns.rename(columns={'Company name' : 'Company',
                         'Regional Commissions':'RC',
                         'CBSA Title':'MSA',
                         'Est. Impact':'Employees'}, inplace=True)

In [64]:
appWarns["Date"] = pd.to_datetime(appWarns["Date"]).dt.strftime('%b %d')

In [65]:
industryRoll = industryClaims[industryClaims['Area'].str.contains("County")]
industryRoll = industryRoll[['Area','Ind','Claims']]
industryRoll = industryRoll.pivot_table('Claims', ['Area'], 'Ind')
industryRoll.reset_index(inplace=True)
industryRoll = industryRoll[~industryRoll['Area'].str.contains(",")]
codes = codes[['area_code','County']]
industryRoll.rename(columns={'Area':'County'}, inplace=True)
industryRoll = pd.merge(industryRoll,codes,how='left',on='County')
industryRoll.rename(columns={'area_code':'GEOID'}, inplace=True)

In [66]:
industryRoll['Total Claims'] = industryRoll.Agriculture + industryRoll.Construction + industryRoll.Education + \
                            industryRoll['Enterprise Management'] + industryRoll.Entertainment + industryRoll.Finance + \
                            industryRoll['Food Services'] + industryRoll['Health Care'] + industryRoll.Information + \
                            industryRoll.Manufacturing + industryRoll['Oil & Gas'] + industryRoll.Other + \
                            industryRoll['Public Admin'] + industryRoll['Real Estate'] + industryRoll.Retail + \
                            industryRoll['Technical Services'] + industryRoll.Transportation + industryRoll.Unclassified + \
                            industryRoll.Utilities + industryRoll['Waste Management'] + industryRoll['Wholesale Trade']

In [67]:
industryRoll['Other Claims'] = industryRoll['Agriculture'] + industryRoll['Oil & Gas'] + industryRoll.Utilities

In [68]:
gaDataCum = dataCum.loc[dataCum['Area'] == 'Georgia']

In [71]:
# write files
# this is the county polygon file for fill layers in mapbox
countyShapes.to_file("app-data/mapbox/countyShapes.geojson", driver='GeoJSON')
# this is the county point file for circle layers in mapbox
countyPoints.to_file("app-data/mapbox/countyPoints.geojson", driver='GeoJSON')
# this is for the github file to draw the d3 charts
data.to_csv('app-data/uiClaims.csv', index=False)
industryClaims.to_csv('app-data/industryClaims.csv', index=False)
industryRoll.to_csv('app-data/countyIndustryClaims.csv', index=False)
dataCum.to_csv('app-data/dataCum.csv', index=False)
gaDataCum.to_csv('app-data/gaDataCum.csv', index=False)
# this is for the warn logs by city for a point layer within the application
appWarns.to_csv('app-data/warns.csv', index=False)
warns.to_excel('app-data/warnsClean.xlsx', index=False)
# this is for the json object to use as dynamically generated KPIs
dataK.to_json(orient='index')

'{"Georgia":{"FebLaborForce":"5.19M","CumClaims":"2.28M","PctFebLabor":"43.9%","CurrentWarns":"363"},"Atlanta-Sandy Springs-Alpharetta, GA":{"FebLaborForce":"3.14M","CumClaims":"1.44M","PctFebLabor":"45.9%","CurrentWarns":"215"},"ARC":{"FebLaborForce":"2.47M","CumClaims":"1.17M","PctFebLabor":"47.3%","CurrentWarns":"188"},"Fulton County":{"FebLaborForce":"566K","CumClaims":"306K","PctFebLabor":"54.1%","CurrentWarns":"97"},"Gwinnett County":{"FebLaborForce":"497K","CumClaims":"221K","PctFebLabor":"44.6%","CurrentWarns":"23"},"Northwest":{"FebLaborForce":"427K","CumClaims":"191K","PctFebLabor":"44.7%","CurrentWarns":"18"},"DeKalb County":{"FebLaborForce":"406K","CumClaims":"184K","PctFebLabor":"45.4%","CurrentWarns":"0"},"Cobb County":{"FebLaborForce":"432K","CumClaims":"172K","PctFebLabor":"39.9%","CurrentWarns":"33"},"Coastal":{"FebLaborForce":"333K","CumClaims":"166K","PctFebLabor":"49.8%","CurrentWarns":"31"},"Northeast":{"FebLaborForce":"309K","CumClaims":"132K","PctFebLabor":"42.6%